# W4-CRISPRi-workflow

Transcriptional interference is another powerful tool for assessing the function of genes, by expressing a guide RNA that targets a dead-Cas9/sgRNA complex to bind to the + (Watson) strand of the promoter and 5’-UTR of the targeted gene, thereby sterically blocking binding and progression of RNA polymerases to the promoter region. Workflow 4 uses CRISPR interference (CRISPRi) with ssDNA bridging to reversibly inactivate genes transcriptionally12,29 (Figure 5A). This approach targets regions upstream of the Transcriptional Start Site (TSS), using a dCas9-sgRNA complex positioned near the TSS (default 100bp upstream) to sterically hinder transcription. This allows for the functional study of genes through controlled knockdown. To get started, users can download the pCRISPR-dCas9 plasmid and the S. coelicolor (A3) genome. StreptoAIM will then generate all necessary components, including primers and plasmids.

In [1]:
import sys
import os
from pydna.dseqrecord import Dseqrecord
from datetime import datetime


# Ensure the src directory is in the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from streptocad.sequence_loading.sequence_loading import (
    load_and_process_plasmid,
    load_and_process_genome_sequences
)
from streptocad.cloning.ssDNA_bridging import assemble_plasmids_by_ssDNA_bridging, make_ssDNA_oligos
from streptocad.crispr.guideRNA_crispri import extract_sgRNAs_for_crispri, SgRNAargs
from streptocad.primers.primer_generation import  primers_to_IDT
from streptocad.utils import generate_project_directory_structure

## INPUT

In [2]:
# Inputs
# 1 Add genome of choice (genbank, fasta)
path_to_genome = '../../data/genomes/Streptomyces_coelicolor_A3_chromosome.gb'
genome = load_and_process_genome_sequences(path_to_genome)[0]

# 2 Add plasmid 
path_to_plasmid = '../../data/plasmids/pCRISPR–Cas9_plasmid_addgene.gbk'
clean_plasmid = load_and_process_plasmid(path_to_plasmid)

# 3 Choose genes to knock out (list)
genes_to_KO = ['SCO5087', 'SCO5089', 'SCO5090']

#### Advanced settings ####
# 4 Filtering metrics for sgRNAs
gc_upper = 0.72
gc_lower = 0.2
off_target_seed = 13
off_target_upper = 0
cas_type='cas9'
number_of_sgRNAs_per_group = 5
extension_to_promoter_region=200

# 6 Choose overlapping sequences for our plasmid we can use the following
#As per the article **"CRISPR–Cas9, CRISPRi and CRISPR-BEST-mediated genetic manipulation in streptomycetes"** we need the following oligoes: 
#CGGTTGGTAGGATCGACGGC **-N20-** GTTTTAGAGCTAGAAATAGC
up_homology = Dseqrecord('CGGTTGGTAGGATCGACGGC')
dw_homology = Dseqrecord('GTTTTAGAGCTAGAAATAGC')

# Computation

In [3]:

# Initialize SgRNAArgs with desired parameters
args = SgRNAargs(path_to_genome, 
                genes_to_KO,
                step=['find', 'filter'],
                gc_upper = gc_upper,
                gc_lower = gc_lower,
                off_target_seed = off_target_seed,
                off_target_upper = off_target_upper,
                cas_type='cas9',
                extension_to_promoter_region=200,
                target_non_template_strand = True
                  )

sgrna_df = extract_sgRNAs_for_crispri(args)
sgrna_df

sgRNA or PAM generated were outside the designated border in SCO5087. Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in SCO5087. Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in SCO5089. Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in SCO5089. Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in SCO5089. Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in SCO5089. Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in SCO5090. Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in SCO5090. Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in SCO5090. Skipping to next locus tag.
sgRNA or PAM generated were outside the designated border in SCO5090. Skipping to next locus tag.
sgRNA or PAM generat

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
11,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-95,0.60,GGG,TCCGAAGTCGCACTGGTGAC,TCGCACTGGTGAC,0
12,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-94,0.60,CGG,CTCCGAAGTCGCACTGGTGA,GTCGCACTGGTGA,0
13,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-88,0.65,TGG,GCAGGACTCCGAAGTCGCAC,TCCGAAGTCGCAC,0
434,Streptomyces_coelicolor_A3,SCO5090,5532706,1,-1,-73,0.65,CGG,CGGGTATGGACACGCCGTAC,GGACACGCCGTAC,0
14,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-70,0.65,AGG,AGGGGAACACATGGCCACGC,CACATGGCCACGC,0
15,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-61,0.65,TGG,GAGGCAGGGAGGGGAACACA,GGAGGGGAACACA,0
351,Streptomyces_coelicolor_A3,SCO5089,5532449,1,-1,-56,0.70,CGG,CCGCCGAGTTGAAGCCCCAG,GTTGAAGCCCCAG,0
16,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-52,0.70,GGG,AGGGACCACGAGGCAGGGAG,ACGAGGCAGGGAG,0
17,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-51,0.70,GGG,GAGGGACCACGAGGCAGGGA,CACGAGGCAGGGA,0
18,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-50,0.70,AGG,TGAGGGACCACGAGGCAGGG,CCACGAGGCAGGG,0


In [4]:
# Filter the DataFrame to retain only up to 5 sgRNA sequences per locus_tag
filtered_df = sgrna_df.groupby('locus_tag').head(number_of_sgRNAs_per_group)
filtered_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
11,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-95,0.60,GGG,TCCGAAGTCGCACTGGTGAC,TCGCACTGGTGAC,0
12,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-94,0.60,CGG,CTCCGAAGTCGCACTGGTGA,GTCGCACTGGTGA,0
13,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-88,0.65,TGG,GCAGGACTCCGAAGTCGCAC,TCCGAAGTCGCAC,0
434,Streptomyces_coelicolor_A3,SCO5090,5532706,1,-1,-73,0.65,CGG,CGGGTATGGACACGCCGTAC,GGACACGCCGTAC,0
14,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-70,0.65,AGG,AGGGGAACACATGGCCACGC,CACATGGCCACGC,0
15,Streptomyces_coelicolor_A3,SCO5087,5529801,1,-1,-61,0.65,TGG,GAGGCAGGGAGGGGAACACA,GGAGGGGAACACA,0
351,Streptomyces_coelicolor_A3,SCO5089,5532449,1,-1,-56,0.70,CGG,CCGCCGAGTTGAAGCCCCAG,GTTGAAGCCCCAG,0
440,Streptomyces_coelicolor_A3,SCO5090,5532706,1,-1,-24,0.60,CGG,ATCAGGTCGAGCAGCTCTCG,CGAGCAGCTCTCG,0
353,Streptomyces_coelicolor_A3,SCO5089,5532449,1,-1,-23,0.60,CGG,GTTACGGGGTCGGTGCGAAA,GGTCGGTGCGAAA,0
354,Streptomyces_coelicolor_A3,SCO5089,5532449,1,-1,-13,0.65,CGG,TCTCGGGGTCGTTACGGGGT,GTCGTTACGGGGT,0


## Output

In [5]:
# MAke oligoes
list_of_ssDNAs = make_ssDNA_oligos(filtered_df, upstream_ovh = up_homology,
                      downstream_ovh=dw_homology)
print(list_of_ssDNAs[0].name)

# cut plasmid
from Bio.Restriction import NcoI
linearized_plasmid = sorted(clean_plasmid.cut(NcoI), key=lambda x: len(x), reverse=True)[0]
#print(linearized_plasmid)

sgRNA_vectors = assemble_plasmids_by_ssDNA_bridging(list_of_ssDNAs,linearized_plasmid)
sgRNA_vectors

SCO5087_loc_-95


[Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279),
 Contig(o11279)]

In [6]:

# Constructing a meaningful name, ID, and description for the assembled plasmid using user input
targeting_info = []
for index, row in filtered_df.iterrows():
    formatted_str = f"CRISPRi_{row['locus_tag']}_p{row['sgrna_loc']}"
    targeting_info.append(formatted_str)

for i in range(len(sgRNA_vectors)):
    sgRNA_vectors[i].name = f'p{targeting_info[i]}_#{i+1}'
    sgRNA_vectors[i].id = sgRNA_vectors[i].name  # Using the same value for ID as for name for simplicity
    sgRNA_vectors[i].description = f'Assembled plasmid targeting {", ".join(genes_to_KO)} for single gene KNOCK-DOWN, assembled using StreptoAIM.'


In [7]:
print_plasmids = False

if print_plasmids: 
    for vector in sgRNA_vectors: 
        vector.write(f"../../data/plasmids/sgRNA_plasmids_pCRISPR–dCas9/{vector.id}.gb")

### IDT primers

In [8]:
idt_primers=primers_to_IDT(list_of_ssDNAs)
idt_primers

,Name,Sequence,Concentration,Purification
0,SCO5087_loc_-95,CGGTTGGTAGGATCGACGGCTCCGAAGTCGCACTGGTGACGTTTTA...,25nm,STD
1,SCO5087_loc_-94,CGGTTGGTAGGATCGACGGCCTCCGAAGTCGCACTGGTGAGTTTTA...,25nm,STD
2,SCO5087_loc_-88,CGGTTGGTAGGATCGACGGCGCAGGACTCCGAAGTCGCACGTTTTA...,25nm,STD
3,SCO5090_loc_-73,CGGTTGGTAGGATCGACGGCCGGGTATGGACACGCCGTACGTTTTA...,25nm,STD
4,SCO5087_loc_-70,CGGTTGGTAGGATCGACGGCAGGGGAACACATGGCCACGCGTTTTA...,25nm,STD
5,SCO5087_loc_-61,CGGTTGGTAGGATCGACGGCGAGGCAGGGAGGGGAACACAGTTTTA...,25nm,STD
6,SCO5089_loc_-56,CGGTTGGTAGGATCGACGGCCCGCCGAGTTGAAGCCCCAGGTTTTA...,25nm,STD
7,SCO5090_loc_-24,CGGTTGGTAGGATCGACGGCATCAGGTCGAGCAGCTCTCGGTTTTA...,25nm,STD
8,SCO5089_loc_-23,CGGTTGGTAGGATCGACGGCGTTACGGGGTCGGTGCGAAAGTTTTA...,25nm,STD
9,SCO5089_loc_-13,CGGTTGGTAGGATCGACGGCTCTCGGGGTCGTTACGGGGTGTTTTA...,25nm,STD


In [9]:


input_files = [
    {"name": "input_genome.gb", "content": genome},
    {"name": "input_plasmid.gb", "content": clean_plasmid}
]

output_files = [
    {"name": "cBEST_w_sgRNAs.gb", "content": sgRNA_vectors}, # LIST OF Dseqrecords
    {"name": "full_idt.csv", "content": idt_primers},
    {"name": "sgrna_df.csv", "content": sgrna_df},
    {"name": "filtered_df.csv", "content": filtered_df}
]

input_values = {
    "genes_to_knockout": genes_to_KO,


    "filtering_metrics": {
        "gc_upper": gc_upper,
        "gc_lower": gc_lower,
        "off_target_seed": off_target_seed,
        "off_target_upper": off_target_upper,
        "cas_type": cas_type,
        "number_of_sgRNAs_per_group": number_of_sgRNAs_per_group,
        'extension_to_promoter_region':extension_to_promoter_region,

    },
    "overlapping_sequences": {
        "up_homology": str(up_homology),
        "dw_homology": str(dw_homology)
    }
}


# Paths to Markdown files
markdown_file_paths = [
    "../../protocols/conjugation_protcol.md",
    "../../protocols/single_target_crispr_plasmid_protcol.md"

]

# Data and time
timestamp = datetime.utcnow().isoformat()

generate_project_directory_structure(
    project_name=f"CRISPRi_workflow_{timestamp}",
    input_files=input_files,
    output_files=output_files,
    input_values=input_values,
    markdown_file_paths=markdown_file_paths

)

Project structure for 'CRISPRi_workflow_2024-06-28T09:32:25.460595' created successfully.


/Users/lucaslevassor/opt/anaconda3/envs/streptocad/lib/python3.11/site-packages/Bio/SeqIO/InsdcIO.py:728: BiopythonWarning: Increasing length of locus line to allow long name. This will result in fields that are not in usual positions.
  warnings.warn(
